# Normalizar e padronizar variáveis

In [1]:
from joblib import dump, load
import pandas as pd

In [2]:
caminho = '../../util/data/'

## seleção das variáveis numéricas

In [3]:
# carregamento das variáveis numericas dos dataframes

df_categorias = load(caminho+'df_categorias.z')
categorias_numerica= df_categorias.select_dtypes(include=['int64', 'float64']) # seleciona apenas as variaveis numericas do tipo int64 e float64

df_clientes = load(caminho+'df_clientes.z')
clientes_numerica = df_clientes.select_dtypes(include=["int64", "float64"])

df_produtos = load(caminho+'df_produtos.z')
produtos_numerica = df_produtos.select_dtypes(include=["int64", "float64"])

# exibe informações das variaveis numericas de cada dataframe
print('[CATEGORIAS]\n',categorias_numerica.describe())
print('[CLIENTES]\n',clientes_numerica.describe())
print('[PRODUTOS]\n',produtos_numerica.describe())

[CATEGORIAS]
            codigo
count   74.000000
mean   303.851351
std    155.563288
min      2.000000
25%    343.000000
50%    381.500000
75%    399.750000
max    419.000000
[CLIENTES]
             codigo     vendedor
count  3132.000000  3132.000000
mean   6786.610153    67.129630
std    2827.491552    30.297795
min       1.000000     7.000000
25%    5194.750000    54.000000
50%    7997.500000    82.000000
75%    8844.250000    89.000000
max    9730.000000   101.000000
[PRODUTOS]
        codigo_categoria   preco_venda
count       9589.000000   9592.000000
mean          81.104390    304.013080
std          130.790131   1366.995745
min            2.000000      0.000000
25%            3.000000     19.735000
50%            8.000000     65.540000
75%           64.000000    214.000000
max          368.000000  51283.640000


com o output, pode-se avaliar:

`categorias`: somente possui uma variável numérica para servir de id da categoria, será descartada

`clientes`: só possui variáveis que servem como identificadores ('codigo','vendedor'), serão descartados

`produtos`: também possui variáveis identificadoras ('codigo_categoria) que será descartada nos procedimentos. será necessário aplicar uma normalização em 'preco_venda' por apresentar valores muito distantes  


# `df_produtos`

In [4]:
# descartando variável codigo_categoria
produtos_numerica = produtos_numerica.drop(columns=['codigo_categoria'])
produtos_numerica.head(10)

,preco_venda
0,0.00
1,8.93
2,0.00
3,8.38
4,67.91
5,441.54
6,14.74
7,26.52
8,954.28
9,16.00


## padronizando valores com o StandardScaler


In [6]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [7]:
minmax = MinMaxScaler()
produtos_numerica_minmax = minmax.fit_transform(produtos_numerica)

In [8]:
scaler = StandardScaler() # declarando o scaler
produtos_numerica_std = scaler.fit_transform(produtos_numerica) # aplicando no produtos

# inserindo a coluna ao lado

In [12]:
df_produtos.columns

Index(['codigo', 'descricao', 'codigo_barras', 'unidade', 'marca',
       'codigo_categoria', 'categoria', 'classificacao', 'preco_venda'],
      dtype='object')

In [ ]:
df_produtos.insert(9, 'preco_venda_minmax', produtos_numerica_minmax)
df_produtos.insert(10, 'preco_venda_std', produtos_numerica_std)
df_produtos[['preco_venda', 'preco_venda_minmax', 'preco_venda_std']].head(10)

,preco_venda,preco_venda_minmax,preco_venda_std
0,0.00,0.000000,-0.222407
1,8.93,0.000174,-0.215874
2,0.00,0.000000,-0.222407
3,8.38,0.000163,-0.216276
4,67.91,0.001324,-0.172726
5,441.54,0.008610,0.100610
6,14.74,0.000287,-0.211623
7,26.52,0.000517,-0.203005
8,954.28,0.018608,0.475715
9,16.00,0.000312,-0.210702


In [16]:
dump(df_produtos, caminho+'df_produtos.z')

['../../util/data/df_produtos.z']

# `df_unidades`

In [18]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [25]:
df_unidades = pd.read_csv("../../util/data/unidades.csv")

In [26]:
# Codificando a coluna 'status' (sim/não) em 1 e 0
df_unidades['status_cod'] = df_unidades['status'].map({'sim': 1, 'não': 0})

In [27]:
# Codificando a coluna 'nome_fantasia' com Label Encoding
le = LabelEncoder()
df_unidades.insert(2, 'nome_fantasia_cod', le.fit_transform(df_unidades['nome_fantasia']))

In [28]:
df_unidades.head()

,codigo_unidade,nome_fantasia,nome_fantasia_cod,status,status_cod
0,10003,L03 - SALVADOR,2,sim,1
1,10002,L02 - SOBRAL,1,sim,1
2,10001,L01 - FORTALEZA,0,sim,1


In [61]:
dump(df_unidades, caminho+'df_unidades.z')

['../../util/data/df_unidades.z']

# `df_vendedores`

In [33]:
from joblib import load
df_vendedores = load("../../util/data/df_vendedores.z")

In [34]:
df_vendedores

,codigo,nome_completo,nome_reduzido
0,11,ADALBERTO MONTENEGRO SILVA,ADALBERTO
1,51,DIOGO MONTEIRO JUNIOR,DIOGO
2,88,PAULO GIRÃO MONTE,PAULO MONTE
3,89,ALEXANDRA SAMPAIO OLIVEIRA,ALEXANDRA
4,54,PATRICIA LEITE NOBRE,PATRICIA
5,56,ALBERTO DAMASCENO NOBREGA,ALBERTO
6,94,MARIO LOPES NASCIMENTO,MARIO LOPES
7,79,VITOR OLIVEIRA SANTOS,VITOR
8,58,CLAUDIA MARIA DAMASCENO,CLAUDIA
9,9,MARCIO SANTOS DE NOBREGA,MARCIO SANTOS


In [35]:
# Aplicando LabelEncoder para nome_completo e nome_reduzido
le_completo = LabelEncoder()
le_reduzido = LabelEncoder()

df_vendedores.insert(2, 'nome_completo_cod', le_completo.fit_transform(df_vendedores['nome_completo']))
df_vendedores.insert(4, 'nome_reduzido_cod', le_reduzido.fit_transform(df_vendedores['nome_reduzido']))

In [37]:
df_vendedores.head(3)

,codigo,nome_completo,nome_completo_cod,nome_reduzido,nome_reduzido_cod
0,11,ADALBERTO MONTENEGRO SILVA,0,ADALBERTO,0
1,51,DIOGO MONTEIRO JUNIOR,4,DIOGO,4
2,88,PAULO GIRÃO MONTE,9,PAULO MONTE,9


In [38]:
dump(df_vendedores, caminho+'df_vendedores.z')

['../../util/data/df_vendedores.z']

# `df_vendas`

In [39]:
from joblib import load
df_vendas = load("../../util/data/df_vendas.z")

In [40]:
print(df_vendas.info())

<class 'pandas.core.frame.DataFrame'>
Index: 10418 entries, 0 to 10427
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   codigo_empresa    10418 non-null  int64         
 1   tipo_pedido       10418 non-null  object        
 2   centro_resultado  10418 non-null  int64         
 3   documento         10418 non-null  int64         
 4   codigo_cliente    10418 non-null  int64         
 5   codigo_empregado  10418 non-null  int64         
 6   data_emissao      10418 non-null  datetime64[ns]
 7   hora_emissao      10418 non-null  object        
 8   data_fechamento   10418 non-null  datetime64[ns]
 9   hora_fechamento   10418 non-null  object        
 10  data_faturamento  10418 non-null  datetime64[ns]
 11  codigo_produto    10418 non-null  int64         
 12  quantidade        10418 non-null  float64       
 13  total             10418 non-null  float64       
 14  custo_medio       10418 non

In [41]:
df_vendas.head(3)

,codigo_empresa,tipo_pedido,centro_resultado,documento,codigo_cliente,codigo_empregado,data_emissao,hora_emissao,data_fechamento,hora_fechamento,data_faturamento,codigo_produto,quantidade,total,custo_medio,valor_desconto
0,2,PV,2,24560,4395,56,2023-03-21,12:10:07,2023-03-21,12:12:40,2023-03-21,194959,8.0,64.0,5.2870,20.64
1,1,PV,2,22433,236,9,2023-02-13,10:40:44,2023-04-27,07:18:40,2023-04-27,195052,6.0,75.0,5.2931,0.00
2,2,PV,2,24560,4395,56,2023-03-21,12:10:07,2023-03-21,12:12:40,2023-03-21,164,3.0,75.0,15.3550,16.80


In [43]:
# Selecionando as colunas numéricas relevantes
num_cols = ['quantidade', 'total', 'custo_medio', 'valor_desconto']

In [44]:
# Inicializando os scalers
minmax = MinMaxScaler()
zscore = StandardScaler()

In [45]:
# Aplicando Min-Max Normalization
df_minmax = pd.DataFrame(
    minmax.fit_transform(df_vendas[num_cols]),
    columns=[col + '_minmax' for col in num_cols],
    index=df_vendas.index
)

In [49]:
df_vendas.insert(13, 'quantidade_minmax', df_minmax['quantidade_minmax'])
df_vendas.insert(15, 'total_minmax', df_minmax['total_minmax'])
df_vendas.insert(17, 'custo_medio_minmax', df_minmax['custo_medio_minmax'])
df_vendas.insert(19, 'valor_desconto_minmax', df_minmax['valor_desconto_minmax'])

In [50]:
# Aplicando Z-score Standardization
df_zscore = pd.DataFrame(
    zscore.fit_transform(df_vendas[num_cols]),
    columns=[col + '_zscore' for col in num_cols],
    index=df_vendas.index
)

In [51]:
df_vendas.columns

Index(['codigo_empresa', 'tipo_pedido', 'centro_resultado', 'documento',
       'codigo_cliente', 'codigo_empregado', 'data_emissao', 'hora_emissao',
       'data_fechamento', 'hora_fechamento', 'data_faturamento',
       'codigo_produto', 'quantidade', 'quantidade_minmax', 'total',
       'total_minmax', 'custo_medio', 'custo_medio_minmax', 'valor_desconto',
       'valor_desconto_minmax'],
      dtype='object')

In [52]:
df_vendas.insert(14, 'quantidade_zscore', df_zscore['quantidade_zscore'])
df_vendas.insert(17, 'total_zscore', df_zscore['total_zscore'])
df_vendas.insert(20, 'custo_medio_zscore', df_zscore['custo_medio_zscore'])
df_vendas.insert(23, 'valor_desconto_zscore', df_zscore['valor_desconto_zscore'])

In [55]:
# One-hot encoding para tipo_pedido
df_vendas = pd.get_dummies(df_vendas, columns=['tipo_pedido'], prefix='pedido')

In [56]:
print(df_vendas.info())

<class 'pandas.core.frame.DataFrame'>
Index: 10418 entries, 0 to 10427
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   codigo_empresa         10418 non-null  int64         
 1   centro_resultado       10418 non-null  int64         
 2   documento              10418 non-null  int64         
 3   codigo_cliente         10418 non-null  int64         
 4   codigo_empregado       10418 non-null  int64         
 5   data_emissao           10418 non-null  datetime64[ns]
 6   hora_emissao           10418 non-null  object        
 7   data_fechamento        10418 non-null  datetime64[ns]
 8   hora_fechamento        10418 non-null  object        
 9   data_faturamento       10418 non-null  datetime64[ns]
 10  codigo_produto         10418 non-null  int64         
 11  quantidade             10418 non-null  float64       
 12  quantidade_minmax      10418 non-null  float64       
 13  quanti

In [58]:
df_vendas.columns

Index(['codigo_empresa', 'centro_resultado', 'documento', 'codigo_cliente',
       'codigo_empregado', 'data_emissao', 'hora_emissao', 'data_fechamento',
       'hora_fechamento', 'data_faturamento', 'codigo_produto', 'quantidade',
       'quantidade_minmax', 'quantidade_zscore', 'total', 'total_minmax',
       'total_zscore', 'custo_medio', 'custo_medio_minmax',
       'custo_medio_zscore', 'valor_desconto', 'valor_desconto_minmax',
       'valor_desconto_zscore', 'pedido_DV', 'pedido_PV'],
      dtype='object')

In [57]:
print(df_vendas.describe())

       codigo_empresa  centro_resultado     documento  codigo_cliente  \
count    10418.000000      10418.000000  10418.000000    10418.000000   
mean         1.583797          4.140910  22114.464293     5790.050490   
min          1.000000          2.000000  18228.000000        2.000000   
25%          1.000000          2.000000  20850.250000     4798.000000   
50%          2.000000          2.000000  22135.000000     6863.000000   
75%          2.000000          7.000000  23375.750000     6863.000000   
max          3.000000         14.000000  24601.000000     9708.000000   
std          0.633558          2.881906   1425.949460     2558.593989   

       codigo_empregado                   data_emissao  \
count      10418.000000                          10418   
mean          63.667691  2023-02-08 16:44:44.277212416   
min            8.000000            2022-12-06 00:00:00   
25%           51.000000            2023-01-19 00:00:00   
50%           63.000000            2023-02-08 00:00:

In [60]:
dump(df_vendas, caminho+'df_vendas.z')

['../../util/data/df_vendas.z']